# Проект по курсу "Python для начинающих"

В качестве проекта я решил составить своебразный рейтинг популярности дистрибутивов Linux на основе частоты их упоминания в подсайте /r/linux на сайте reddit.com. Для извлечения комментариев я использовал PRAW (Python Reddit API Wrapper), после чего проверял, имеется ли в тексте комментария название какого-либо дистрибутива Linux. Список дистрибутивов был взят с сайта distrowatch.com. 

## 1. Извлечение комментариев

Для извлечения комментариев я использовал PRAW: https://praw.readthedocs.io/en/stable/index.html. Так как получение десятков тысяч комментариев может занять длительное время, я ограничился 150 последними "горячими" (hot) темами подсайта /r/linux. Чтобы проверить работоспособность программы без необходимости регистрации аккаунта разработчика на реддите и скачивания нескольких тысяч комментариев, сохраняем комментарии в файл comments_file.

In [13]:
import praw
from praw.models import MoreComments

reddit = praw.Reddit(client_id='my cliend id',
                     client_secret='my client secret',
                     user_agent='my user agent')
 
comments_file = r"/home/user/reddit_linux/comments_file"
count = 0

with open(comments_file, 'w') as f:
    for submission in reddit.subreddit("linux").hot(limit=150):
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            count += 1
            f.write(f"{comment.body}\n")
            if count % 500 == 0:
                print(f"{count} comments extracted")

print(f"Finished. {count} comments extracted")

500 comments extracted
1000 comments extracted
1500 comments extracted
2000 comments extracted
2500 comments extracted
3000 comments extracted
3500 comments extracted
4000 comments extracted
4500 comments extracted
5000 comments extracted
5500 comments extracted
6000 comments extracted
6500 comments extracted
7000 comments extracted
7500 comments extracted
8000 comments extracted
8500 comments extracted
Finished. 8518 comments extracted


## 2. Получение списка дистрибутивов

Копируем список дистрибутивов с distrowatch.com и приводим в пригодный для дальнейшей работы вид: извлекаем название дистрибутива между символами ">" и "<", обрезаем в случае необходимости пробелы, переводим в нижний регистр и помещаем в множество dists.

In [1]:
dists_string = """<option value="3cx">3CX</option><option value="4mlinux">4MLinux</option><option value="absolute">Absolute</option><option value="academix">AcademiX</option><option value="alma">AlmaLinux</option><option value="alpine">Alpine</option><option value="alt">ALT</option><option value="anarchy">Anarchy</option><option value="antix">antiX</option><option value="apodio">APODIO</option><option value="arch">Arch</option><option value="archbang">ArchBang</option><option value="archcraft">Archcraft</option><option value="archlabs">ArchLabs</option><option value="archman">Archman</option><option value="archstrike">ArchStrike</option><option value="arco">ArcoLinux</option><option value="armbian">Armbian</option><option value="artix">Artix</option><option value="avlinux">AV Linux</option><option value="baruwa">Baruwa</option><option value="batocera">Batocera</option><option value="bedrock">Bedrock</option><option value="berry">Berry</option><option value="bicom">Bicom</option><option value="biglinux">BigLinux</option><option value="blackarch">BlackArch</option><option value="blueonyx">BlueOnyx</option><option value="bluestar">Bluestar</option><option value="bodhi">Bodhi</option><option value="boss">BOSS</option><option value="bsdrp">BSDRP</option><option value="bunsenlabs">BunsenLabs</option><option value="cae">CAELinux</option><option value="caine">CAINE</option><option value="calculate">Calculate</option><option value="centos">CentOS</option><option value="clear">Clear</option><option value="clearos">ClearOS</option><option value="clonezilla">Clonezilla</option><option value="cloudready">CloudReady</option><option value="condres">Condres</option><option value="crux">CRUX</option><option value="daphile">Daphile</option><option value="debian">Debian</option><option value="skolelinux">Debian Edu</option><option value="deepin">deepin</option><option value="devuan">Devuan</option><option value="diamond">Diamond</option><option value="dietpi">DietPi</option><option value="dragonflybsd">DragonFly</option><option value="easynas">EasyNAS</option><option value="easyos">EasyOS</option><option value="elastix">Elastix</option><option value="elearnix">eLearnix</option><option value="elementary">elementary</option><option value="elive">Elive</option><option value="emmabuntus">Emmabuntüs</option><option value="endeavour">EndeavourOS</option><option value="endian">Endian</option><option value="endless">Endless</option><option value="enso">Enso</option><option value="eurolinux">EuroLinux</option><option value="exe">Exe</option><option value="exherbo">Exherbo</option><option value="extix">ExTiX</option><option value="fatdog">Fatdog64</option><option value="fedora">Fedora</option><option value="ferenos">Feren</option><option value="finnix">Finnix</option><option value="freebsd">FreeBSD</option><option value="freedombox">FreedomBox</option><option value="freepbx">FreePBX</option><option value="freespire">Freespire</option><option value="fuguita">FuguIta</option><option value="funtoo">Funtoo</option><option value="fwul">FWUL</option><option value="garuda">Garuda</option><option value="gecko">Gecko</option><option value="gentoo">Gentoo</option><option value="ghostbsd">GhostBSD</option><option value="gparted">GParted</option><option value="grml">Grml</option><option value="guixsd">Guix System</option><option value="haiku">Haiku</option><option value="hamonikr">HamoniKR</option><option value="hardenedbsd">HardenedBSD</option><option value="hyperbola">Hyperbola</option><option value="ipfire">IPFire</option><option value="jingos">JingOS</option><option value="kaisen">Kaisen</option><option value="kali">Kali</option><option value="kanotix">KANOTIX</option><option value="kaos">KaOS</option><option value="karoshi">Karoshi</option><option value="kdeneon">KDE neon</option><option value="kiss">KISS</option><option value="knoppix">KNOPPIX</option><option value="kodachi">Kodachi</option><option value="kolibri">KolibriOS</option><option value="kubuntu">Kubuntu</option><option value="lakka">Lakka</option><option value="lfs">LFS</option><option value="libreelec">LibreELEC</option><option value="linspire">Linspire</option><option value="linuxfx">Linuxfx</option><option value="lite">Lite</option><option value="liveraizo">Live Raizo</option><option value="lliurex">LliureX</option><option value="lockbox">LockBox</option><option value="lubuntu">Lubuntu</option><option value="lxle">LXLE</option><option value="mabox">Mabox</option><option value="mageia">Mageia</option><option value="makulu">MakuluLinux</option><option value="manjaro">Manjaro</option><option value="max">MAX</option><option value="midnightbsd">MidnightBSD</option><option value="mint">Mint</option><option value="miracle">Miracle</option><option value="murena">Murena</option><option value="mx">MX Linux</option><option value="navy">Navy</option><option value="neptune">Neptune</option><option value="netbsd">NetBSD</option><option value="netrunner">Netrunner</option><option value="nexentastor">NexentaStor</option><option value="nitrux">Nitrux</option><option value="nixos">NixOS</option><option value="nomadbsd">NomadBSD</option><option value="nova">Nova</option><option value="nst">NST</option><option value="nutyx">NuTyX</option><option value="ob2d">OB2D</option><option value="obarun">Obarun</option><option value="omarine">Omarine</option><option value="omoikane">Omoikane</option><option value="openbsd">OpenBSD</option><option value="openindiana">OpenIndiana</option><option value="openmamba">openmamba</option><option value="openmandriva">OpenMandriva</option><option value="openmediavault">OpenMediaVault</option><option value="opensuse">openSUSE</option><option value="opnsense">OPNsense</option><option value="oracle">Oracle</option><option value="osgeo">OSGeoLive</option><option value="osmc">OSMC</option><option value="ovios">OviOS</option><option value="pakos">PakOS</option><option value="paldo">paldo</option><option value="parabola">Parabola</option><option value="pardus">Pardus</option><option value="parrot">Parrot</option><option value="partedmagic">Parted Magic</option><option value="pclinuxos">PCLinuxOS</option><option value="pearl">Pearl</option><option value="pentoo">Pentoo</option><option value="peppermint">Peppermint</option><option value="peropesis">Peropesis</option><option value="pfsense">pfSense</option><option value="photonos">Photon</option><option value="pisi">Pisi</option><option value="plamo">Plamo</option><option value="pld">PLD</option><option value="plop">Plop</option><option value="popos">Pop!_OS</option><option value="porteus">Porteus</option><option value="porteuskiosk">Porteus Kiosk</option><option value="postmarketos">postmarketOS</option><option value="primtux">PrimTux</option><option value="proxmox">Proxmox</option><option value="puppy">Puppy</option><option value="pureos">PureOS</option><option value="q4os">Q4OS</option><option value="qubes">Qubes</option><option value="raspios">RasPiOS</option><option value="rasplex">RasPlex</option><option value="rds">RDS</option><option value="reactos">ReactOS</option><option value="rebeccablackos">RebeccaBlackOS</option><option value="rebornos">RebornOS</option><option value="recalbox">Recalbox</option><option value="redhat">Red Hat</option><option value="redcore">Redcore</option><option value="redo">Redo</option><option value="refracta">Refracta</option><option value="regata">Regata</option><option value="remnux">REMnux</option><option value="rescatux">Rescatux</option><option value="rescuezilla">Rescuezilla</option><option value="risc">RISC</option><option value="robolinux">Robolinux</option><option value="rockstor">Rockstor</option><option value="rocky">Rocky</option><option value="rosa">ROSA</option><option value="rss">RSS</option><option value="runtu">Runtu</option><option value="salient">Salient</option><option value="salix">Salix</option><option value="scientific">Scientific</option><option value="selks">SELKS</option><option value="septor">Septor</option><option value="sharklinux">Shark</option><option value="siduction">siduction</option><option value="slackel">Slackel</option><option value="slackware">Slackware</option><option value="slax">Slax</option><option value="slitaz">SliTaz</option><option value="smartos">SmartOS</option><option value="smeserver">SME Server</option><option value="snal">Snal</option><option value="solaris">Solaris</option><option value="solus">Solus</option><option value="solydxk">SolydXK</option><option value="sophos">Sophos</option><option value="sourcemage">Source Mage</option><option value="sparky">SparkyLinux</option><option value="springdale">Springdale</option><option value="star">Star</option><option value="supergrub">Super Grub2</option><option value="sle">SUSE</option><option value="swift">Swift</option><option value="syslinuxos">SysLinuxOS</option><option value="systemrescue">SystemRescue</option><option value="t2">T2</option><option value="tails">Tails</option><option value="thinstation">Thinstation</option><option value="tinycore">Tiny Core</option><option value="tooppy">ToOpPy</option><option value="trisquel">Trisquel</option><option value="truenas">TrueNAS</option><option value="turnkey">TurnKey</option><option value="ubos">UBOS</option><option value="ubports">UBports</option><option value="ubuntu">Ubuntu</option><option value="ubuntubudgie">Ubuntu Budgie</option><option value="ubuntuce">Ubuntu Christian</option><option value="ubuntudp">Ubuntu DP</option><option value="ubuntukylin">Ubuntu Kylin</option><option value="ubuntumate">Ubuntu MATE</option><option value="ubuntustudio">Ubuntu Studio</option><option value="ufficiozero">Ufficio Zero</option><option value="ultimate">Ultimate</option><option value="ultramarine">Ultramarine</option><option value="univention">Univention</option><option value="untangle">Untangle</option><option value="uruk">Uruk</option><option value="venom">Venom</option><option value="vine">Vine</option><option value="void">Void</option><option value="volumio">Volumio</option><option value="voyager">Voyager</option><option value="vyos">VyOS</option><option value="vz">VzLinux</option><option value="whonix">Whonix</option><option value="wifislax">Wifislax</option><option value="xigmanas">XigmaNAS</option><option value="xstreamos">XStreamOS</option><option value="xubuntu">Xubuntu</option><option value="yunohost">YunoHost</option><option value="zentyal">Zentyal</option><option value="zenwalk">Zenwalk</option><option value="zephix">Zephix</option><option value="zevenet">Zevenet</option><option value="zorin">Zorin</option>            </select>
"""

dists = set()

for i in range(len(dists_string)):
    if i < len(dists_string) - 1 and dists_string[i] == ">":
        start = i + 1
        finish = start
        while finish < len(dists_string) and dists_string[finish] != "<":
            finish += 1
        s = dists_string[start:finish].strip().lower()
        if s:
            dists.add(s)

print(dists)


{'gentoo', 'guix system', 'karoshi', 'puppy', 'pentoo', 'kali', 'alt', 'dietpi', 'kodachi', 'ubuntu mate', 'volumio', 'haiku', 'libreelec', 'rockstor', 'freespire', 'makululinux', 'lxle', 'caine', 'lfs', 'murena', 'archbang', 'osmc', 'academix', 'springdale', 'absolute', 'bodhi', 'clear', 'cloudready', 'bicom', 'risc', 'condres', 'runtu', 'emmabuntüs', 'diamond', 'devuan', 'jingos', 'remnux', 'opensuse', 'vzlinux', 'elementary', 'nomadbsd', 'mabox', 'opnsense', 'pakos', 'anarchy', 'almalinux', 'robolinux', 'armbian', 'arch', 'raspios', 'debian edu', 'max', 'rss', 'ubos', 'ubuntu studio', 'whonix', 'zenwalk', 'omoikane', 'trisquel', 'exe', 'gparted', 'hyperbola', 'biglinux', 'selks', 'primtux', 'batocera', 'fatdog64', 'archlabs', 'av linux', 'xubuntu', 'nst', 'septor', 'void', 'snal', 'kolibrios', 'nitrux', 'univention', 'finnix', 'slitaz', 'sme server', 'fwul', 'knoppix', 'slax', 'solydxk', 'midnightbsd', 'fuguita', 'blackarch', 'endeavouros', 'miracle', 'rds', '3cx', 'caelinux', 'rasp

## 3. Подсчет упоминаний дистрибутивов в комментариях

Создаем словарь для хранения количества упоминаний для дистрибутивов, после чего построчно считываем комментарии из файла comments_file, разбиваем на отдельные слова, проверяем наличие слова в сете dists и, если оно там есть, увеличиваем счетчик для этого слова на единицу в словаре. После этого сортируем элементы словаря по убыванию значений и выводим первые 20 результатов.

In [3]:
d = {}
comments_file = r"/home/user/reddit_linux/comments_file"

with open(comments_file, 'r') as f:
    for post in f:
        for word in post.split(" "):
            word = word.lower().strip(",.!?;:'")
            if word in dists:
                d[word] = d.get(word, 0) + 1
sorted_dists = sorted(d.items(), key=lambda item: item[1], reverse=True)

for i in range(min(len(sorted_dists), 20)):
    print(f"{i+1}. {sorted_dists[i][0]}: {sorted_dists[i][1]}")

1. ubuntu: 577
2. fedora: 448
3. arch: 426
4. centos: 402
5. debian: 348
6. bedrock: 213
7. mint: 134
8. rocky: 93
9. opensuse: 72
10. gentoo: 67
11. clear: 54
12. zorin: 53
13. slackware: 52
14. nixos: 47
15. manjaro: 43
16. suse: 36
17. freebsd: 25
18. elementary: 24
19. openbsd: 23
20. deepin: 20
